In [1]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors



In [3]:
def get_lat_long(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(address, latitude, longitude)
    return latitude, longitude
get_lat_long("new york")

new york 40.7127281 -74.0060152


(40.7127281, -74.0060152)

In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
def saving_data(area_name, latitude, longitude):
    url = f"https://api.foursquare.com/v2/venues/explore?client_id=N40W0THAJDZYKLHLHWRDBU01LIMNXBMXZ03X5ZOGZSRVMLSR&client_secret=GZNJSGG3SLAYGMN4BJCC5VIOFBT2EW10OUF1V20ZO4P3V2YT&ll={latitude},{longitude}&v=20180604&radius=30000&limit=1000"
    results = requests.get(url).json()
    dataframe = pd.json_normalize(results["response"]["groups"][0]["items"])
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1) # category for each row
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns] # clean columns
    
    dataframe_filtered.to_pickle(f"{area_name}.pkl") # save to pickle so that wont call the api over and over again
    return dataframe_filtered

In [8]:
saving_data("new york", 40.7127281, -74.0060152)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,The Bar Room at Temple Court,Hotel Bar,123 Nassau St,40.711448,-74.006802,"[{'label': 'display', 'lat': 40.7114477287544,...",157,10038,US,Financial District,New York,NY,United States,"[123 Nassau St, New York, NY 10038, United Sta...",NaN,57f0689d498e7d49d9189369
1,Aire Ancient Baths,Spa,88 Franklin St,40.718141,-74.004941,"[{'label': 'display', 'lat': 40.71814141164123...",609,10013,US,NaN,New York,NY,United States,"[88 Franklin St (at Church St), New York, NY 1...",at Church St,4fbbd9ede4b0756c0d4c2364
2,9/11 Memorial North Pool,Memorial Site,West St,40.712077,-74.013187,"[{'label': 'display', 'lat': 40.71207726282092...",609,NaN,US,NaN,New York,NY,United States,"[West St (at Fulton St), New York, NY, United ...",at Fulton St,58801864cc5b6a14dedce689
3,Crown Shy,Restaurant,70 Pine St,40.706187,-74.007490,"[{'label': 'display', 'lat': 40.70618693053086...",738,10005,US,NaN,New York,NY,United States,"[70 Pine St, New York, NY 10005, United States]",NaN,5c883f65f4b525002c0bf2ca
4,Washington Market Park,Playground,Greenwich St,40.717046,-74.011095,"[{'label': 'display', 'lat': 40.71704598853704...",643,NaN,US,NaN,New York,NY,United States,"[Greenwich St (at Reade St), New York, NY, Uni...",at Reade St,4a9bcc2ff964a5203b3520e3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Greenlight Bookstore,Bookstore,686 Fulton St,40.686356,-73.974560,"[{'label': 'display', 'lat': 40.68635635950454...",3957,11217,US,NaN,Brooklyn,NY,United States,"[686 Fulton St (at S Portland Ave), Brooklyn, ...",at S Portland Ave,4ad937aef964a520281921e3
96,L'Industrie Pizzeria,Pizza Place,254 S 2nd St,40.711606,-73.957802,"[{'label': 'display', 'lat': 40.711606, 'lng':...",4070,11211,US,NaN,Brooklyn,NY,United States,"[254 S 2nd St (Havemeyer), Brooklyn, NY 11211,...",Havemeyer,55135426498e9dee8f19cc5f
97,Evelina Restaurant,Italian Restaurant,211 Dekalb Ave,40.689629,-73.971018,"[{'label': 'display', 'lat': 40.689629, 'lng':...",3916,11205,US,NaN,Brooklyn,NY,United States,"[211 Dekalb Ave (Adelphi), Brooklyn, NY 11205,...",Adelphi,5a00f3538496ca58fd9e2856
98,Ample Hills Creamery,Ice Cream Shop,305 Nevins St,40.678958,-73.987269,"[{'label': 'display', 'lat': 40.67895768082717...",4078,11215,US,NaN,Brooklyn,NY,United States,"[305 Nevins St (at Union St), Brooklyn, NY 112...",at Union St,53c057e7498e1923339d49c8
